In [1]:
## Importo librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [7]:
## leo los dos archivos
trainlimpio= pd.read_csv('train_con_Eda')
test= pd.read_parquet ('test.parquet')

In [8]:
#### Realizo con test, las misma operaciones en train

## Borro las columnas lat y long
borrar= ['lat', 'long']
test.drop(columns= borrar, inplace= True)
## Completo los NaN para el modelo
test['laundry_options'] = test.laundry_options.fillna(test.laundry_options.mode()[0])
test['parking_options'] = test.parking_options.fillna(test.parking_options.mode()[0])
## Borro las columnas que no son para el analisis
Otro_borrar= ['id', 'url', 'region_url', 'image_url', 'description', 'state']
test.drop(columns= Otro_borrar, inplace= True)
## Cambio el tipo de dato
test['baths']=test['baths'].astype (int)

In [9]:
#### Ahora trabajo con las categoricas de test

categoricas= ['type', 'laundry_options', 'parking_options', 'region']
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

#Veo las categoricas
print(test[categoricas].head())

# CreoLabelEncoder : le
le = LabelEncoder()

# Aplico LabelEncoder a las columnas
test[categoricas] = test[categoricas].apply(lambda x: le.fit_transform(x))

# Veo las 5 filas de las categoricas
print(test[categoricas].head())

           type  laundry_options     parking_options                   region
0  manufactured      w/d in unit  off-street parking                ann arbor
1     apartment      w/d hookups  off-street parking                  vermont
2     apartment      w/d in unit             carport                ann arbor
3     apartment      w/d in unit     attached garage  fort collins / north CO
4     apartment  laundry on site  off-street parking          charlottesville
   type  laundry_options  parking_options  region
0     8                4                4       9
1     0                3                4     370
2     0                4                1       9
3     0                4                0     112
4     0                1                4      53


In [10]:
trainlimpio.shape
#print (test.shape)

(148880, 14)

In [11]:
#### ahora escalamos los datos numericos de ambos

from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
scl.fit(trainlimpio[['sqfeet']]) 

trainlimpio['sqfeet'] = scl.transform(trainlimpio[['sqfeet']]) 


from sklearn.preprocessing import StandardScaler
scl = StandardScaler()
scl.fit(test[['sqfeet']]) 

test['sqfeet'] = scl.transform(test[['sqfeet']]) 

#### Comienzo con el modelo; utilizan segun el análisi que voy a desarrollar, todas las columnas

In [28]:
## Declaro X e Y para hacer un train_test_split

X= trainlimpio[['type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed','wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'region']]
y= trainlimpio[['category_price']]

In [29]:
# Separamos los datos
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [30]:
## Vamos a optimizar hiperparámetros
# Grilla para random search

import scipy as sp
from scipy import stats

param_dist = {'n_neighbors':sp.stats.randint(1, 20),
              'weights': ['uniform', 'distance'], 
              'leaf_size':sp.stats.randint(1, 10),
              'algorithm':['auto', 'kd_tree']}

In [31]:
## Instancio y entreno el modelo

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV

knn = KNeighborsClassifier()
model = RandomizedSearchCV(knn, param_dist,n_iter=100, random_state=0, cv=5)

# Entrenamos knn con la grilla
model.fit(X_train, y_train)

C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sha

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=100,
                   param_distributions={'algorithm': ['auto', 'kd_tree'],
                                        'leaf_size': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002663E9CCDC8>,
                                        'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000266350AAEC8>,
                                        'weights': ['uniform', 'distance']},
                   random_state=0)

In [32]:
## Observo los mejores parámetros

print("Mejores hiperparámetros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

scores = pd.DataFrame(model.cv_results_)
scores

Mejores hiperparámetros: {'algorithm': 'auto', 'leaf_size': 4, 'n_neighbors': 6, 'weights': 'distance'}
Mejor Score: 0.8734215592215534



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.892827,0.037938,1.314047,0.086890,auto,6,1,distance,"{'algorithm': 'auto', 'leaf_size': 6, 'n_neigh...",0.864993,0.866336,0.863440,0.865413,0.868346,0.865706,0.001618,60
1,0.909010,0.085741,2.266156,0.204442,kd_tree,4,8,distance,"{'algorithm': 'kd_tree', 'leaf_size': 4, 'n_ne...",0.872801,0.873809,0.871500,0.871248,0.874265,0.872725,0.001203,7
2,0.837331,0.040085,3.509951,0.220391,kd_tree,6,19,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 6, 'n_ne...",0.832627,0.831157,0.832417,0.828555,0.838791,0.832709,0.003369,98
3,0.944712,0.059611,3.595371,0.256735,kd_tree,7,13,uniform,"{'algorithm': 'kd_tree', 'leaf_size': 7, 'n_ne...",0.842366,0.841694,0.842156,0.837077,0.847271,0.842113,0.003232,86
4,1.048696,0.174762,2.311487,0.342888,kd_tree,7,8,distance,"{'algorithm': 'kd_tree', 'leaf_size': 7, 'n_ne...",0.872885,0.873725,0.871542,0.871332,0.874223,0.872741,0.001150,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.815582,0.051130,2.271862,0.162930,auto,6,12,distance,"{'algorithm': 'auto', 'leaf_size': 6, 'n_neigh...",0.873431,0.872382,0.870450,0.871626,0.874601,0.872498,0.001433,14
96,0.870549,0.009019,2.538882,0.195300,auto,3,14,distance,"{'algorithm': 'auto', 'leaf_size': 3, 'n_neigh...",0.873389,0.871836,0.870157,0.870870,0.875273,0.872305,0.001837,20
97,0.787783,0.031309,2.228531,0.099264,kd_tree,7,11,distance,"{'algorithm': 'kd_tree', 'leaf_size': 7, 'n_ne...",0.873053,0.871878,0.870115,0.871122,0.874307,0.872095,0.001465,29
98,1.021509,0.067736,3.234932,0.503344,auto,1,4,uniform,"{'algorithm': 'auto', 'leaf_size': 1, 'n_neigh...",0.858066,0.859494,0.857773,0.858150,0.863644,0.859425,0.002191,68


#### Ahora utilizo el test para predecir

In [33]:
## Declaro mi x_test que tiene las mismas columnas que el train
X_test= test[['type', 'sqfeet', 'beds', 'baths', 'cats_allowed', 'dogs_allowed', 'smoking_allowed','wheelchair_access', 'electric_vehicle_charge', 'comes_furnished', 'laundry_options', 'parking_options', 'region']]

In [34]:
## Predigo
prediction = model.predict(X_test)

In [37]:
## Obtengo un dataframe
y_pred = pd.DataFrame(prediction, columns=['pred'])

In [38]:
y_pred

,pred
0,0
1,1
2,0
3,0
4,0
...,...
38493,0
38494,0
38495,1
38496,0


In [39]:
## Exporto los resultados

y_pred.to_csv('SoleGon6.csv', index=False)